In [8]:
import sqlite3
import seaborn
import pandas as pd

In [17]:
conn = sqlite3.connect("../data_aggregation/options_database.db")
cursor = conn.cursor()

query = """
select 
    contract_data.*,
    price_data.close as stock_price
from contract_data
join price_data
on (price_data.Symbol = contract_data.Symbol) and (price_data.date_of_close = contract_data.obs_date)
    where
        contract_data.symbol ='SPY'
        and obs_date between '2023-05-01' and '2023-06-21'
;
"""
cursor.execute(query);

data = cursor.fetchall()

In [20]:
df = pd.DataFrame(data, columns=['symbol', 'contract_type', 'strike', 'bid', 'midpoint', 'ask', 'last', 'volume', 'open_int', 'obs_date', 'exp_date', 'stock_price'])
df.head()

,symbol,contract_type,strike,bid,midpoint,ask,last,volume,open_int,obs_date,exp_date,stock_price
0,SPY,call,280.0,133.21,133.40,133.58,133.95,0,0.0,2023-05-05,2023-06-02,412.63
1,SPY,call,285.0,128.25,128.42,128.60,0.00,0,0.0,2023-05-05,2023-06-02,412.63
2,SPY,call,290.0,123.27,123.45,123.63,0.00,0,0.0,2023-05-05,2023-06-02,412.63
3,SPY,call,295.0,118.28,118.47,118.66,0.00,0,0.0,2023-05-05,2023-06-02,412.63
4,SPY,call,300.0,113.32,113.50,113.69,106.16,0,8.0,2023-05-05,2023-06-02,412.63


In [31]:
display(df.obs_date.value_counts().head(4))
ddf = df[df.obs_date == '2023-05-12'].copy()
ddf =ddf.sort_values(by=['strike', 'contract_type'])

2023-05-12    618
2023-05-19    614
2023-06-16    570
2023-05-26    504
Name: obs_date, dtype: int64

In [35]:
def out_of_money(x):
    if x.contract_type == 'call':
        if(x.strike > x.stock_price):
            return True
    else:
        if(x.strike < x.stock_price):
            return True
    return False

ddf[ddf[["contract_type", "strike", "stock_price"]].apply(out_of_money, axis=1)]

,symbol,contract_type,strike,bid,midpoint,ask,last,volume,open_int,obs_date,exp_date,stock_price
922,SPY,put,150.0,0.00,0.00,0.01,0.01,159,64782.0,2023-05-12,2023-06-16,411.59
923,SPY,put,160.0,0.00,0.00,0.01,0.01,1,17267.0,2023-05-12,2023-06-16,411.59
924,SPY,put,170.0,0.01,0.02,0.02,0.01,0,13384.0,2023-05-12,2023-06-16,411.59
925,SPY,put,180.0,0.01,0.02,0.02,0.01,1,9169.0,2023-05-12,2023-06-16,411.59
719,SPY,put,185.0,0.01,0.02,0.02,0.01,0,5329.0,2023-05-12,2023-06-16,411.59
...,...,...,...,...,...,...,...,...,...,...,...,...
714,SPY,call,695.0,0.00,0.00,0.01,0.01,0,1440.0,2023-05-12,2023-06-16,411.59
715,SPY,call,700.0,0.00,0.00,0.01,0.01,0,1081.0,2023-05-12,2023-06-16,411.59
716,SPY,call,705.0,0.00,0.00,0.01,0.01,0,1228.0,2023-05-12,2023-06-16,411.59
717,SPY,call,710.0,0.00,0.00,0.01,0.01,0,141.0,2023-05-12,2023-06-16,411.59


In [36]:
ddf['exp_date'].unique()

array(['2023-06-16', '2023-06-09'], dtype=object)

## NO PANDAS CALCS

In [137]:
def my_row_factory(cur, row):
    d={}
    for idx, col in enumerate(cur.description):
        d[col[0].lower()] = row[idx]
    return d

conn = sqlite3.connect("../data_aggregation/options_database.db")
conn.row_factory = my_row_factory
cursor = conn.cursor()

query = """
select 
    contract_data.*,
    price_data.close as stock_price
from contract_data
join price_data
on (price_data.Symbol = contract_data.Symbol) and (price_data.date_of_close = contract_data.obs_date)
    where
        contract_data.symbol ='SPY'
        and obs_date = '2022-05-20'
;
"""
# '2023-05-12'


cursor.execute(query);
data = cursor.fetchall()
# conn.close()

In [138]:
len(data)

1736

In [135]:
%%time

dt = filter(out_of_money, data)

calls = filter(lambda x: x['contract_type']=='call', dt)
puts = filter(lambda x: x['contract_type']=='put', dt)

calls = sorted(calls, key=lambda x: abs(x['strike'] - x['stock_price']))
puts = sorted(puts, key=lambda x: abs(x['strike'] - x['stock_price']))

calls2 = filter_serious(calls)
puts2 = filter_serious(puts)

CPU times: total: 0 ns
Wall time: 2 ms


In [139]:
%%time
data = [opt for opt in data if out_of_money(opt)]

calls = [opt for opt in data if (opt['contract_type'] == 'call')]
puts = [opt for opt in data if (opt['contract_type'] == 'put')]

calls = sorted(calls, key=lambda x: abs(x['strike'] - x['stock_price']))
puts = sorted(puts, key=lambda x: abs(x['strike'] - x['stock_price']))

calls2 = filter_serious(calls)
puts2 = filter_serious(puts)

CPU times: total: 0 ns
Wall time: 2 ms


In [140]:
len(puts2)

345

In [84]:
def out_of_money(x):
    if x['contract_type'] == 'call':
        if(x['strike'] > x['stock_price']):
            return True
    else:
        if(x['strike'] < x['stock_price']):
            return True
    return False

def check_serious(bid, ask):
    return (bid > 0 and ask > 0)

def filter_serious(ls):
    ## Takes in a list of options sorted by ascending difference between strike and stock_price.
    new_ls = []
    prev_serious = True
    for opt in ls:
        if check_serious(opt['bid'], opt['ask']):
            new_ls.append(opt)
            prev_serious = True
        else:
            if not prev_serious:
                break
            prev_serious = False
            
    return new_ls
            

In [76]:
data

[{'symbol': 'SPY',
  'contract_type': 'call',
  'strike': 412.0,
  'bid': 6.77,
  'midpoint': 6.78,
  'ask': 6.78,
  'last': 6.82,
  'volume': 2071,
  'open_int': 5379.0,
  'obs_date': '2023-05-12',
  'exp_date': '2023-06-09',
  'stock_price': 411.59},
 {'symbol': 'SPY',
  'contract_type': 'call',
  'strike': 412.5,
  'bid': 6.44,
  'midpoint': 6.45,
  'ask': 6.46,
  'last': 6.47,
  'volume': 3101,
  'open_int': 3560.0,
  'obs_date': '2023-05-12',
  'exp_date': '2023-06-09',
  'stock_price': 411.59},
 {'symbol': 'SPY',
  'contract_type': 'call',
  'strike': 413.0,
  'bid': 6.13,
  'midpoint': 6.14,
  'ask': 6.15,
  'last': 6.3,
  'volume': 1523,
  'open_int': 3977.0,
  'obs_date': '2023-05-12',
  'exp_date': '2023-06-09',
  'stock_price': 411.59},
 {'symbol': 'SPY',
  'contract_type': 'call',
  'strike': 414.0,
  'bid': 5.53,
  'midpoint': 5.54,
  'ask': 5.54,
  'last': 5.67,
  'volume': 2202,
  'open_int': 2389.0,
  'obs_date': '2023-05-12',
  'exp_date': '2023-06-09',
  'stock_price'